In [3]:
import nltk
from nltk.corpus import stopwords
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer
from sklearn.linear_model import LogisticRegression
import re

In [4]:
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', -1)

In [5]:
stopwords = stopwords.words('english')[:]

In [6]:
pd.set_option("display.max_rows",270)

# Task A

### Engagement Score

In [7]:
post = pd.read_csv("insta_data_with_labels.csv").iloc[:,1:]

In [8]:
max_like = max(post["likes"])
max_like

1539774

In [9]:
max_like = max(post["likes"])
list_norm_like = []
for p in post.itertuples():
    norm_like = p.likes/max_like
    list_norm_like.append(norm_like)
post["norm_like"] = list_norm_like

max_comment = max(post["comments"])
list_norm_comment = []
for p in post.itertuples():
    norm_comment = p.comments/max_comment
    list_norm_comment.append(norm_comment)
    
post["norm_comment"]= list_norm_comment

In [10]:
engagement_score = []
for p in post.itertuples():
    e_score = .4*p.norm_like+.6*p.norm_comment
    engagement_score.append(e_score)
engagement_score

[0.017560655627206684,
 0.02366236449064101,
 0.037809338825375745,
 0.09019406690899491,
 0.11252898280760024,
 0.12070288926685988,
 0.0684502432182017,
 0.07439259702505385,
 0.029396026017428026,
 0.044150234902194656,
 0.08526724827039955,
 0.06025193407719019,
 0.12698927056147763,
 0.15885322135378185,
 0.06036176590398917,
 0.062475533746834275,
 0.07939817644979734,
 0.10823153463623596,
 0.06637011537807715,
 0.0524855773452004,
 0.09030157519960061,
 0.08801654248682615,
 0.07918353096459323,
 0.10909306582838439,
 0.33221843192878076,
 0.08145705246775684,
 0.04558561823412439,
 0.12271770638568008,
 0.058085794017089146,
 0.0878135163021513,
 0.032037474945001886,
 0.03397843335917866,
 0.03537797494481368,
 0.05954897098807081,
 0.26121185644432376,
 0.24268638545740612,
 0.14122382089357513,
 0.1482207042829119,
 0.09477811178034269,
 0.043401602872195966,
 0.1697252296637716,
 0.05805284878448901,
 0.02543701615277202,
 0.040006916819220216,
 0.05253045494082848,
 0.097

In [11]:
med_e_score = np.median(engagement_score)

In [12]:
eng_score = []
for i in range(len(engagement_score)):
    if engagement_score[i]>=med_e_score:
        e_score = 1
    else:
        e_score = 0
    eng_score.append(e_score)
eng_score
post["engagement_score"] = eng_score

In [13]:
post

,Unnamed: 0.1,urls,captions,comments,likes,labels,norm_like,norm_comment,engagement_score
0,0,https://scontent-sea1-1.cdninstagram.com/vp/7831adc9c2d624937ec8197b4affcded/5C76DA0B/t51.2885-15/e15/c181.0.718.718/s480x480/44904753_194498844802328_8238377254064471543_n.jpg?ig_cache_key=MTkxMjcyNTE4Njk1Njg2MzM4Mg%3D%3D.2.c,"Photo by @amivitale | Ntipiyon Nonguta and her son Bernard, one, relax with their neighbors as they make beaded belts for theLoisaba Community Trust in Ewaso Village in Laikipia, Kenya. These women are in a community surrounding Loisaba Conservancy (@loisaba_conservancy) and are being supported and given access to international markets. Revenue from ecotourism is reinvested into neighboring communities like #Ewaso . To get involved and learn more about community based conservation, please follow @loisaba_conservancy and @amivitale. @nature_org @natgeoimagecollection #kenya #magicalkenya #whyilovekenya #northernkenya #africa #children #mothersandsons #mothers #empowerment #conservation #everydayafrica #photojournalism #amivitale",278,61459,"['people', 'public space', 'community', 'tribe', 'tree', 'travel', 'temple', 'recreation', 'tourism', 'crowd']",0.039914,0.002658,0
1,1,https://scontent-sea1-1.cdninstagram.com/vp/53f7539662906933bbbce050cd4fdc2c/5BEF2CD7/t51.2885-15/e15/s640x640/45543088_970491189813847_3800824331312248168_n.jpg,"Video by @amivitale | Wild adult elephants drink and cool off at a watering on the Namunyak Wildlife Conservancy. An elephant’s trunk typically holds around half of a gallon of water, but a big bull elephant can hold as much as 2.5 gallons. After spraying themselves, the elephants will often coat themselves with dust which acts as a bug repellent and sunblock. Namunyak Wildlife Conservancy is home to Reteti Elephant Sanctuary (@r.e.s.c.u.e), the first ever community-owned and run elephant sanctuary in Africa. Reteti operates in partnership with Conservation International (@conservationorg) who provide critical operational support and work to scale the Reteti community-centered model to create lasting impacts worldwide. Follow @amivitale @conservationorg & @r.e.s.c.u.e to learn about the incredible work being done to protect and preserve these magical beings. @nrt_kenya @kenyawildlifeservice @sandiegozoo @tusk_org @thephotosociety @natgeo #namunyak #protectelephants #bekindtoelephants #DontLetThemDisappear #elephants #saveelephants #stoppoaching #kenya #northernkenya #magicalkenya #whyilovekenya #africa #everydayafrica #natureisspeaking #photojournalism #amivitale",362,83092,NaN,0.053964,0.003461,0
2,2,https://scontent-sea1-1.cdninstagram.com/vp/1e4ff18aa09db5069b6e1c6e9e65eac0/5C6F3508/t51.2885-15/e15/s480x480/44577672_336304220507048_1738524933735107944_n.jpg?ig_cache_key=MTkxMjQyNTAwNDk3MzMzNTE0NQ%3D%3D.2,"Photo by George Steinmetz @geosteinmetz | Epic flying last week over the Pyramids of Giza. I had tried for years, and was turned down even with Egyptian military helicopters. Last week Egypt hosted the world paramotor championships, and suddenly the impossible happened. This is me, the last one to land at sunset. For a rare view of the pyramids from above, go to @geosteinmetz",431,136026,"['air sports', 'paragliding', 'landscape', 'sand', 'ecoregion', 'desert', 'powered paragliding', 'windsports', 'sahara', 'steppe']",0.088342,0.004121,0
3,3,https://scontent-sea1-1.cdninstagram.com/vp/a43b38e630c9b7b76c93d76517e4882e/5C8A9361/t51.2885-15/e15/s480x480/42697363_1856252364473290_1258265168604898264_n.jpg?ig_cache_key=MTkxMjM1MDIwNDMwMDk5NzU5Mg%3D%3D.2,"Photo by @irablockphoto | A nighttime view from the top of the Old Town Bridge Tower of the Charles Bridge and Prague Castle along the banks of the Vltava River in Prague, Czech Republic. The Charles Bridge was constructed under King Charles IV of Bohemia starting in 1357 and was finished in the begging of the 15th century. The bridge is a very popular tourist destination especially at sunset and early evening when the city lights up

# Task B

## Logistic regression

### Labels

In [14]:
post.dropna(axis=0, how='any', inplace=True)

In [15]:
label=post["labels"].values.tolist()
label=" ".join(label)
label = label.lower()        

In [16]:
# tokenize the text 
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
labels = tokenizer.tokenize(label)

In [17]:
filter_labels = Counter(x for x in labels if x not in stopwords and x.isdigit() == False and len(x) != 1 and x.isalpha() == True)

In [18]:
filter_labels.most_common()[:100]

[('sky', 78),
 ('wildlife', 48),
 ('tree', 46),
 ('mammal', 45),
 ('water', 44),
 ('fauna', 39),
 ('mountain', 30),
 ('photography', 30),
 ('grass', 30),
 ('ecosystem', 29),
 ('wilderness', 28),
 ('animal', 28),
 ('organism', 27),
 ('rock', 24),
 ('terrestrial', 24),
 ('recreation', 22),
 ('nature', 22),
 ('girl', 22),
 ('plant', 21),
 ('elephant', 21),
 ('landscape', 20),
 ('bird', 19),
 ('darkness', 18),
 ('snout', 18),
 ('sea', 17),
 ('fun', 17),
 ('temple', 16),
 ('phenomenon', 16),
 ('cloud', 16),
 ('reflection', 15),
 ('landforms', 15),
 ('atmosphere', 15),
 ('human', 15),
 ('forest', 15),
 ('geological', 14),
 ('highland', 14),
 ('park', 14),
 ('black', 14),
 ('night', 14),
 ('grassland', 14),
 ('mountainous', 13),
 ('marine', 13),
 ('national', 13),
 ('formation', 13),
 ('city', 12),
 ('reserve', 12),
 ('like', 12),
 ('site', 12),
 ('hill', 12),
 ('ice', 12),
 ('ecoregion', 11),
 ('monochrome', 11),
 ('horizon', 11),
 ('morning', 11),
 ('area', 11),
 ('tourism', 10),
 ('eye', 1

In [219]:
natural_objects=["wilderness","organism","autumn","spring","reflections","phenomenon","wood","nature","natrual",'penguins','cattle','species','insect','flower', 'whiskers','amphibian','fauna','elephant','animal','whale','bird','reptile','fish','grass','tree','plant','bird','cats','dog','horse','lions','bear','mammoths','elephants','wildlife','mammal',"tiger","snake"]
human = ["immigrants","population","dancer","male","female",'women','refugee',"snout",'chest','eye','hair','face','child','human',"family","children",'girl',"girls","people",'photojournalism',"community","communityies",'migrants',"man"]
geological  = ["desert","national park",'formation', 'geological','terrain','badlands',"landscape",'mountains',"mountain"'wilderness','forest','highland','grassland','hill','steppe','jungle',"vallay",'horizon','ecosystem','rock','terrestrial','landforms','landform','ecoregion',"geology","hillside"]
aerial =['cloud','sky','sunrise', 'star','atmosphere','aerial','light','darkness', 'sunset','night','astronomical' ,"sun","sunlight"]
activity = ["swimming","marriage","workshop",'festival','cycling','music','fun','climbing','religion','carnival','flame','ceremony','wedding', 'ritual','travel',"paragliding",'game', "recreation",'event','tradition','safari','vacation','sports','sport','competition event', 'boating',"boat"]
artificial_objects = ["pyramids","conservation","artificial",'painting','photography','sanctuary','aircraft',"food",'house','metropolis','outerwear', 'wall','design','room','arts','village','hall','fashion accessory','history', 'furniture','infrastructure', 'campfire','window','building','temple','city','street','vehicle','architecture','leisure','tourism','town','urban','computer','structure',"castle","cities"'town','landmark','town square', 'plaza', 'tourist attraction', 'road',"robots","transportation" ]
water=['island',"islands",'ocean','sea','iceberg', 'arctic ocean', 'shore','sea',"wave",'ice', 'shore','arctic','glacial ', 'polar ice cap', 'ice', 'melting', 'sea', 'ice cap','river','underwater',"coast","snow"]

In [161]:
def replacement(text):
    
    text = text.lower()
    
    for w in human:
        try:
            text = text.replace(w," human ")
        except:
            pass   
    for w in natural_objects:
        try:
            text = text.replace(w, ' natural ')
        except:
            pass
    for w in geological:
        try:
            text = text.replace(w, ' geological ')
        except:
            pass
    for w in aerial:
        try:
            text = text.replace(w, ' aerial ')
        except:
            pass
    for w in activity:
        try:
            text = text.replace(w, ' activity ')
        except:
            pass
    for w in water:
        try:
            text = text.replace(w, ' water ')
        except:
            pass

    for w in artificial_objects:
        try:
            text = text.replace(w, ' artificial ')
        except:
            pass
  

    return text

In [220]:
label_column = []
for p in post['labels']:
    new_text =replacement(p)
    label_column.append(new_text)

In [23]:
len(post['labels'])

238

In [24]:
len(post["engagement_score"])

238

In [221]:
def cal_dfitf_score(text,min_df):
    vectorizer=CountVectorizer(stop_words="english", analyzer='word', min_df=min_df, max_features=None)
    transformer=TfidfTransformer()
    tfidf=transformer.fit_transform(vectorizer.fit_transform(text))
    word=vectorizer.get_feature_names()
    weight=tfidf.toarray()
    a=[]
    for i in range(len(weight)):
        print (u"-------post",i,u"tf-idf------")
        a.append(weight[i])
        for j in range(len(word)):
            print (word[j],weight[i][j])
    return a,word



In [222]:
a,word=cal_dfitf_score(label_column,0.1)

-------post 0 tf-idf------
activity 0.5443541075823809
aerial 0.0
artificial 0.4207342989062119
geological 0.0
hu 0.5071738058535387
human 0.4939068575891619
natural 0.15966214944226542
water 0.0
-------post 1 tf-idf------
activity 0.8051805649299142
aerial 0.0
artificial 0.0
geological 0.5930297276352545
hu 0.0
human 0.0
natural 0.0
water 0.0
-------post 2 tf-idf------
activity 0.0
aerial 0.22839125783986483
artificial 0.9235733224067173
geological 0.0
hu 0.0
human 0.0
natural 0.0
water 0.3079768684186239
-------post 3 tf-idf------
activity 0.0
aerial 0.2797088583446032
artificial 0.2827731258753374
geological 0.8083811068953415
hu 0.0
human 0.0
natural 0.21461604247211077
water 0.37717668827026035
-------post 4 tf-idf------
activity 0.0
aerial 0.0
artificial 0.0
geological 0.0
hu 0.0
human 0.0
natural 0.9434193001794524
water 0.33160220754529396
-------post 5 tf-idf------
activity 0.0
aerial 0.24664884948842977
artificial 0.0
geological 0.9504466949143439
hu 0.0
human 0.0
natural 0.1

hu 0.0
human 0.0
natural 0.6013708146149745
water 0.0
-------post 73 tf-idf------
activity 0.0
aerial 0.0
artificial 0.0
geological 0.0
hu 0.3472931924554375
human 0.33820849453985335
natural 0.8746441863381912
water 0.0
-------post 74 tf-idf------
activity 0.0
aerial 0.9298309038498768
artificial 0.18800347816885157
geological 0.0
hu 0.22662863423420157
human 0.22070035022010212
natural 0.0
water 0.0
-------post 75 tf-idf------
activity 0.0
aerial 0.0
artificial 0.0
geological 0.0
hu 0.0
human 0.0
natural 1.0
water 0.0
-------post 76 tf-idf------
activity 0.0
aerial 0.0
artificial 0.0
geological 0.22373658017762335
hu 0.28302781963096846
human 0.27562421282579413
natural 0.8909929793828474
water 0.0
-------post 77 tf-idf------
activity 0.0
aerial 0.0
artificial 0.8516947946183727
geological 0.0
hu 0.3422249240080281
human 0.33327280481497595
natural 0.21546998811534332
water 0.0
-------post 78 tf-idf------
activity 0.0
aerial 0.2843392137393027
artificial 0.0
geological 0.958723740985

activity 0.0
aerial 0.6998619153570603
artificial 0.23584301507629296
geological 0.6742190828450871
hu 0.0
human 0.0
natural 0.0
water 0.0
-------post 126 tf-idf------
activity 0.0
aerial 0.0
artificial 0.29981804567787085
geological 0.28570282611076303
hu 0.0
human 0.0
natural 0.9102104342613444
water 0.0
-------post 127 tf-idf------
activity 0.0
aerial 0.8292444861487783
artificial 0.5588860189625867
geological 0.0
hu 0.0
human 0.0
natural 0.0
water 0.0
-------post 128 tf-idf------
activity 0.0
aerial 0.08195757906228542
artificial 0.0
geological 0.0
hu 0.0
human 0.0
natural 0.06288471295847395
water 0.9946499223899732
-------post 129 tf-idf------
activity 0.0
aerial 0.9848728822141768
artificial 0.0
geological 0.13554104169793174
hu 0.0
human 0.0
natural 0.10795384149987396
water 0.0
-------post 130 tf-idf------
activity 0.0
aerial 0.0
artificial 0.0
geological 0.3860680464083028
hu 0.0
human 0.0
natural 0.9224703049651389
water 0.0
-------post 131 tf-idf------
activity 0.5514279820

aerial 0.0
artificial 0.19432800639202238
geological 0.0
hu 0.7027575943439511
human 0.684374451249902
natural 0.0
water 0.0
-------post 201 tf-idf------
activity 0.0
aerial 0.0
artificial 0.0
geological 0.0
hu 0.1871749681978032
human 0.18227873619448137
natural 0.9427855271493744
water 0.20711215176606743
-------post 202 tf-idf------
activity 0.0
aerial 0.0
artificial 0.0
geological 0.31899796769382555
hu 0.4035339200742968
human 0.3929780443986258
natural 0.7622131779084588
water 0.0
-------post 203 tf-idf------
activity 0.9241635846140656
aerial 0.0
artificial 0.3571455716336324
geological 0.0
hu 0.0
human 0.0
natural 0.13553121240425312
water 0.0
-------post 204 tf-idf------
activity 0.22343167227715358
aerial 0.0
artificial 0.0
geological 0.32912275712254246
hu 0.0
human 0.0
natural 0.9174728871021152
water 0.0
-------post 205 tf-idf------
activity 0.0
aerial 0.3269901198380819
artificial 0.0
geological 0.9450277570147222
hu 0.0
human 0.0
natural 0.0
water 0.0
-------post 206 tf-

In [223]:
df=pd.DataFrame(a)
df.columns=word

In [224]:
df

,activity,aerial,artificial,geological,hu,human,natural,water
0,0.544354,0.000000,0.420734,0.000000,0.507174,0.493907,0.159662,0.000000
1,0.805181,0.000000,0.000000,0.593030,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.228391,0.923573,0.000000,0.000000,0.000000,0.000000,0.307977
3,0.000000,0.279709,0.282773,0.808381,0.000000,0.000000,0.214616,0.377177
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.943419,0.331602
5,0.000000,0.246649,0.000000,0.950447,0.000000,0.000000,0.189250,0.000000
6,0.385742,0.000000,0.894427,0.000000,0.000000,0.000000,0.226281,0.000000
7,0.265003,0.202603,0.819288,0.000000,0.000000,0.000000,0.466361,0.000000
8,0.000000,0.000000,0.000000,0.176546,0.000000,0.000000,0.984292,0.000000
9,0.000000,0.787009,0.000000,0.505448,0.000000,0.000000,0.000000,0.353750


In [225]:
df.drop(['hu'],axis=1,inplace=True)

In [226]:
y=post["engagement_score"]

In [227]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
def logictic_pred(df,y):
    X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=0)
    lr = LogisticRegression() 
    lr.fit(X_train, y_train)
    y_pred=lr.predict(X_test)
    print("confusion_matrix:\n",confusion_matrix(y_test, y_pred))
    print("accuracy_score:\n",accuracy_score(y_test, y_pred))

In [228]:
logictic_pred(df,y)

confusion_matrix:
 [[14  9]
 [ 2 23]]
accuracy_score:
 0.7708333333333334


### Captions

In [171]:
text=post["captions"].values.tolist()
text=" ".join(text)
text = text.lower()
tokens = tokenizer.tokenize(text)

In [172]:
filter_words = Counter(x for x in tokens if x not in stopwords and x.isdigit() == False and len(x) != 1 and x.isalpha() == True)

In [173]:
filter_words.most_common()[:300]

[('photo', 260),
 ('one', 89),
 ('see', 80),
 ('people', 80),
 ('follow', 77),
 ('years', 74),
 ('world', 73),
 ('natgeo', 65),
 ('many', 57),
 ('year', 56),
 ('like', 54),
 ('first', 53),
 ('wildlife', 50),
 ('time', 49),
 ('work', 47),
 ('life', 46),
 ('national', 45),
 ('new', 44),
 ('species', 41),
 ('part', 41),
 ('old', 40),
 ('around', 40),
 ('park', 40),
 ('back', 39),
 ('story', 38),
 ('china', 36),
 ('way', 35),
 ('still', 34),
 ('long', 34),
 ('wild', 34),
 ('women', 33),
 ('get', 32),
 ('conservation', 32),
 ('land', 32),
 ('made', 32),
 ('animals', 31),
 ('last', 29),
 ('day', 29),
 ('make', 28),
 ('also', 28),
 ('found', 28),
 ('kenya', 27),
 ('africa', 27),
 ('today', 27),
 ('images', 27),
 ('nature', 27),
 ('across', 27),
 ('find', 27),
 ('city', 25),
 ('war', 25),
 ('poaching', 25),
 ('live', 25),
 ('project', 25),
 ('assignment', 25),
 ('since', 25),
 ('children', 24),
 ('home', 24),
 ('elephants', 24),
 ('red', 24),
 ('water', 23),
 ('visit', 23),
 ('along', 22),
 ('

In [48]:
post[post['captions'].str.contains('wilderness')]

,Unnamed: 0.1,urls,captions,comments,likes,labels,norm_like,norm_comment,engagement_score
62,62,https://scontent-sea1-1.cdninstagram.com/vp/370380772687360d05c5bb68b1677ad4/5C6CD345/t51.2885-15/fr/e15/s1080x1080/44748360_724455554590987_6045188723392899316_n.jpg?ig_cache_key=MTkwNzM2NTY4Mjg0MTcxNzEwOA%3D%3D.2,"Photo by Michael Melford @michaelmelford | The Allagash River in Maine under moonlight. Allagash means “bark stream” in the native Penobscot language; in the image is a handmade birch bark canoe. This year we celebrate the 50th anniversary of the Wild and Scenic Rivers Act, which preserved certain stretches of our rivers. I am honored to have shot images for five U.S. postal stamps, to be issued next year, of designated Wild and Scenic Rivers in recognition of the importance of preserving these wilderness areas. #rivers #wilderness #maine #Allagash #preservation #nature #moonlight",991,418235,"['reflection', 'water', 'nature', 'sky', 'body of water', 'river', 'lake', 'calm', 'loch', 'horizon']",0.271621,0.009476,1
164,164,https://scontent-sea1-1.cdninstagram.com/vp/ac8b0ac06e14096c781dd36bfb13b594/5C7C4BBE/t51.2885-15/e35/c166.0.667.667/s480x480/43778687_284348615748113_3746736055572920724_n.jpg?ig_cache_key=MTg5NzM1OTE0MTU1MjAzNzU4MA%3D%3D.2.c,"Photo by @katieorlinsky // #sponsored by @t._rowe_price // I have been working in Alaska for close to five years, and even though a lot of that work includes following trails across thousands and thousands of miles, I have only explored a tiny fraction of the largest state. Yet somehow this photo feels as familiar to me as a snapshot of the Brooklyn Bridge. I often fly along this same route through these same mountains in the same bush plane. From way up high, I fantasize about how I would know exactly where I was if I found myself in the valleys thousands of feet below. The reality of course is that I have no idea what lies beneath me in the beautiful and brutal landscape of the Alaska Range, or for that matter the majority of Alaska's approximate 50 million square miles of wilderness. // T. Rowe Price goes beyond to gain an understanding of what lies beneath the surface of investment opportunities. Check out today's story to learn more and visit natgeo.com/gobeyond",486,262649,"['sky', 'aerial photography', 'fell', 'arctic', 'highland', 'ice cap', 'mountain', 'glacial landform', 'atmosphere', 'geological phenomenon']",0.170576,0.004647,0


In [154]:
def clean_data(text):
    word=[]
    #delete punctuations stopwords
    words = re.compile(r'[^a-zA-Z]+').split(text)
    for w in words:
        if w not in stopwords and len(w)>2:
            word.append(w)
    return word

In [229]:
caption_column = []
for p in post['captions']:
    a=clean_data(p)
    a= ",".join(a)
    new_text =replacement(a) 
    caption_column.append(new_text)

In [230]:
b,w=cal_dfitf_score(caption_column,0.2)

-------post 0 tf-idf------
activity 0.0
aerial 0.0
artificial 0.4233411994170296
follow 0.15685087325433122
geological 0.0
hu 0.6618168571032561
human 0.5833456649626321
natgeo 0.0
natural 0.11238069983373727
photo 0.07250443823419964
water 0.0
world 0.0
years 0.0
-------post 1 tf-idf------
activity 0.0
aerial 0.4111354497932118
artificial 0.5789143061624155
follow 0.0
geological 0.0
hu 0.0
human 0.0
natgeo 0.0
natural 0.0
photo 0.1982979998726716
water 0.0
world 0.491202657693869
years 0.46392210592382876
-------post 2 tf-idf------
activity 0.0
aerial 0.5965450227677515
artificial 0.629990277153067
follow 0.0
geological 0.0
hu 0.0
human 0.0
natgeo 0.0
natural 0.0
photo 0.0719310904157866
water 0.33923401657551017
world 0.35636005210077504
years 0.0
-------post 3 tf-idf------
activity 0.0
aerial 0.0
artificial 0.0
follow 0.0
geological 0.5459589780075325
hu 0.0
human 0.0
natgeo 0.0
natural 0.0
photo 0.2539390248667271
water 0.7984007552493336
world 0.0
years 0.0
-------post 4 tf-idf---

natgeo 0.15318708613797977
natural 0.7951325797638278
photo 0.06412426811585316
water 0.20161085567835502
world 0.3176835968230276
years 0.0
-------post 47 tf-idf------
activity 0.0
aerial 0.0
artificial 0.3238388848182651
follow 0.47993828096486835
geological 0.0
hu 0.3375087411814772
human 0.5949811007140665
natgeo 0.0
natural 0.0
photo 0.44370368779549474
water 0.0
world 0.0
years 0.0
-------post 48 tf-idf------
activity 0.15197479232109137
aerial 0.0
artificial 0.11079970309488932
follow 0.0
geological 0.0
hu 0.0
human 0.203569529160109
natgeo 0.0
natural 0.9412170186938305
photo 0.07590539489635038
water 0.0
world 0.0
years 0.1775821777017719
-------post 49 tf-idf------
activity 0.0
aerial 0.0
artificial 0.14181639323513157
follow 0.0
geological 0.0
hu 0.7390136670361178
human 0.6513892996025595
natgeo 0.0
natural 0.0
photo 0.09715395466420972
water 0.0
world 0.0
years 0.0
-------post 50 tf-idf------
activity 0.0
aerial 0.0
artificial 0.30893780734018284
follow 0.0
geological 0.0


hu 0.425529475492752
human 0.5358210339100562
natgeo 0.1909146937672713
natural 0.6193508898053381
photo 0.039958541281219895
water 0.1256322440173621
world 0.09898103706294811
years 0.09348380030424372
-------post 80 tf-idf------
activity 0.0
aerial 0.0
artificial 0.0
follow 0.0
geological 0.0
hu 0.0
human 0.0
natgeo 0.0
natural 0.0
photo 0.5367286866159136
water 0.8437548915197803
world 0.0
years 0.0
-------post 81 tf-idf------
activity 0.1956221856863416
aerial 0.0
artificial 0.1426215477039469
follow 0.0
geological 0.21006309356695288
hu 0.14864187498029008
human 0.1310175050257769
natgeo 0.0
natural 0.4543259960829283
photo 0.0977055406901727
water 0.767981383720155
world 0.0
years 0.22858405141008228
-------post 82 tf-idf------
activity 0.0
aerial 0.0
artificial 0.0
follow 0.42512810785280875
geological 0.4225012862342665
hu 0.0
human 0.0
natgeo 0.4694584978582465
natural 0.0
photo 0.19651579871956487
water 0.6178584099016305
world 0.0
years 0.0
-------post 83 tf-idf------
activi

world 0.0
years 0.0
-------post 114 tf-idf------
activity 0.3571436416757993
aerial 0.7396744119077779
artificial 0.26038140178076297
follow 0.0
geological 0.38350812795066236
hu 0.0
human 0.0
natgeo 0.0
natural 0.0
photo 0.17837911631322523
water 0.2804177523717198
world 0.0
years 0.0
-------post 115 tf-idf------
activity 0.10197427435444184
aerial 0.0
artificial 0.1486920185805167
follow 0.11018286428261631
geological 0.0
hu 0.6973586639959187
human 0.6829703367160498
natgeo 0.0
natural 0.0
photo 0.05093211476669644
water 0.0
world 0.0
years 0.0
-------post 116 tf-idf------
activity 0.0
aerial 0.0
artificial 0.33109120952874393
follow 0.0981372857882581
geological 0.09753090587798666
hu 0.4140806404036742
human 0.6083056159012963
natgeo 0.0
natural 0.5625081519965783
photo 0.045364036732962694
water 0.0
world 0.0
years 0.10613006418559616
-------post 117 tf-idf------
activity 0.0
aerial 0.0
artificial 0.9689278334836233
follow 0.0
geological 0.0
hu 0.0
human 0.0
natgeo 0.0
natural 0.

human 0.19243423228272677
natgeo 0.0
natural 0.0
photo 0.14350670705248852
water 0.676791584244586
world 0.0
years 0.0
-------post 156 tf-idf------
activity 0.21184137648538773
aerial 0.0
artificial 0.3088928270183415
follow 0.2288939027269027
geological 0.682438774526622
hu 0.0
human 0.14188027043539794
natgeo 0.0
natural 0.0
photo 0.10580638467685002
water 0.49899319815376997
world 0.0
years 0.24753613667807728
-------post 157 tf-idf------
activity 0.5246515680691989
aerial 0.0
artificial 0.6693851321945092
follow 0.14172106858859043
geological 0.0
hu 0.09966302200788664
human 0.3513841705383181
natgeo 0.31299817041267236
natural 0.10154047942943843
photo 0.0655106742523825
water 0.10298490322124661
world 0.0
years 0.0
-------post 158 tf-idf------
activity 0.22225805152795397
aerial 0.15343835026182773
artificial 0.16204086048504668
follow 0.08004969515515391
geological 0.0
hu 0.056293637984994746
human 0.04961893812626018
natgeo 0.08839690659682366
natural 0.9176656095170433
photo 0

-------post 199 tf-idf------
activity 0.0
aerial 0.0
artificial 0.33227162680556716
follow 0.0
geological 0.0
hu 0.3462974452756125
human 0.3052371835515898
natgeo 0.0
natural 0.7056420307161233
photo 0.07587628348284708
water 0.0
world 0.37590527515799055
years 0.17751407097216523
-------post 200 tf-idf------
activity 0.17302600829053247
aerial 0.1791756817040406
artificial 0.0
follow 0.0
geological 0.0
hu 0.5258894373851312
human 0.6953026058243579
natgeo 0.4128974528985037
natural 0.0
photo 0.08641964424523416
water 0.0
world 0.0
years 0.0
-------post 201 tf-idf------
activity 0.4778235378658591
aerial 0.7422094427641868
artificial 0.0
follow 0.0
geological 0.25654833113904957
hu 0.0
human 0.0
natgeo 0.0
natural 0.0
photo 0.11932697448880054
water 0.3751717429153575
world 0.0
years 0.0
-------post 202 tf-idf------
activity 0.0
aerial 0.0
artificial 0.0
follow 0.16290515747264847
geological 0.1618985837328351
hu 0.11456038580633371
human 0.10097703574532087
natgeo 0.17989215276010215

In [231]:
df1=pd.DataFrame(b)
df1.columns=w

In [232]:
df1.drop(["hu","follow","years"],axis=1,inplace=True)

In [233]:
df1

,activity,aerial,artificial,geological,human,natgeo,natural,photo,water,world
0,0.000000,0.000000,0.423341,0.000000,0.583346,0.000000,0.112381,0.072504,0.000000,0.000000
1,0.000000,0.411135,0.578914,0.000000,0.000000,0.000000,0.000000,0.198298,0.000000,0.491203
2,0.000000,0.596545,0.629990,0.000000,0.000000,0.000000,0.000000,0.071931,0.339234,0.356360
3,0.000000,0.000000,0.000000,0.545959,0.000000,0.000000,0.000000,0.253939,0.798401,0.000000
4,0.000000,0.226642,0.000000,0.235019,0.439749,0.000000,0.677736,0.109313,0.343688,0.000000
5,0.000000,0.000000,0.000000,0.349348,0.326835,0.000000,0.000000,0.081245,0.766321,0.402504
6,0.000000,0.000000,0.656562,0.000000,0.301571,0.000000,0.000000,0.224895,0.000000,0.557086
7,0.000000,0.000000,0.858458,0.210733,0.262871,0.000000,0.000000,0.098017,0.308172,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.461267,0.273918,0.710899,0.114662,0.000000,0.000000
9,0.000000,0.868482,0.305724,0.000000,0.000000,0.000000,0.000000,0.209442,0.329250,0.000000


In [234]:
logictic_pred(df1,y)

confusion_matrix:
 [[17  6]
 [ 7 18]]
accuracy_score:
 0.7291666666666666


### labels+captions

In [235]:
post["all"]=post['captions']+post['labels']

In [236]:
all_column = []
for p in post['all']:
    a=clean_data(p)
    a= ",".join(a)
    new_text =replacement(a) 
    all_column.append(new_text)

In [237]:
c,z=cal_dfitf_score(all_column,0.2)

-------post 0 tf-idf------
activity 0.19117184524568798
aerial 0.0
artificial 0.4232863716924154
follow 0.11992315481862875
geological 0.0
hu 0.6328654466223437
human 0.5717629181826791
like 0.13834847294427668
natgeo 0.0
national 0.0
natural 0.14232220566778944
photo 0.05543457164754758
water 0.0
world 0.0
years 0.0
-------post 1 tf-idf------
activity 0.6631528440600897
aerial 0.1490278874211096
artificial 0.24472184600801633
follow 0.0
geological 0.6045912979560052
hu 0.0
human 0.0
like 0.0
natgeo 0.0
national 0.0
natural 0.0
photo 0.09614803320038878
water 0.0
world 0.22968896919423862
years 0.22494023172901312
-------post 2 tf-idf------
activity 0.0
aerial 0.46078375581230074
artificial 0.7566627514101124
follow 0.0
geological 0.0
hu 0.0
human 0.0
like 0.0
natgeo 0.0
national 0.0
natural 0.0
photo 0.059456592479033434
water 0.361811185879306
world 0.28407286106099755
years 0.0
-------post 3 tf-idf------
activity 0.0
aerial 0.20811581608856258
artificial 0.17087569171782954
follow 0

-------post 41 tf-idf------
activity 0.13734263058056134
aerial 0.0
artificial 0.20273300426787563
follow 0.0
geological 0.0
hu 0.6819995261144373
human 0.6161531512449424
like 0.1987859999630271
natgeo 0.0
national 0.0
natural 0.0
photo 0.0796511629146658
water 0.12117529995089928
world 0.0
years 0.1863454763153476
-------post 42 tf-idf------
activity 0.3115634056236366
aerial 0.0
artificial 0.0
follow 0.0
geological 0.09468331244640353
hu 0.25785401819731174
human 0.23295846960278568
like 0.15031614213537245
natgeo 0.0
national 0.1538110174828967
natural 0.85048489820612
photo 0.06022987296970352
water 0.0
world 0.0
years 0.0
-------post 43 tf-idf------
activity 0.0
aerial 0.0
artificial 0.0
follow 0.1240227408230658
geological 0.2703720079469989
hu 0.08181250253654539
human 0.07391358691452933
like 0.0
natgeo 0.0
national 0.2928090323185656
natural 0.8831250338735116
photo 0.057329608468684866
water 0.17443392533271845
world 0.0
years 0.0
-------post 44 tf-idf------
activity 0.62151

follow 0.0
geological 0.0
hu 0.594208501453644
human 0.53683826256173
like 0.0
natgeo 0.0
national 0.3544478959239469
natural 0.1781717635868977
photo 0.13879598545827115
water 0.0
world 0.0
years 0.0
-------post 72 tf-idf------
activity 0.1780222200197609
aerial 0.4800753057373354
artificial 0.0
follow 0.0
geological 0.649205901962499
hu 0.0
human 0.0
like 0.0
natgeo 0.0
national 0.0
natural 0.5301306785740795
photo 0.1032430847529679
water 0.15706627882131646
world 0.0
years 0.0
-------post 73 tf-idf------
activity 0.0
aerial 0.0
artificial 0.0
follow 0.0
geological 0.0
hu 0.4028988623476006
human 0.3639993785374292
like 0.0
natgeo 0.33722912664630555
national 0.0
natural 0.7248486079483693
photo 0.14116445112132273
water 0.21475699890730354
world 0.0
years 0.0
-------post 74 tf-idf------
activity 0.0
aerial 0.6935086179562822
artificial 0.11388251698768254
follow 0.0
geological 0.0
hu 0.38310398890796593
human 0.5768594673494005
like 0.0
natgeo 0.0
national 0.0
natural 0.0
photo 0.0

-------post 105 tf-idf------
activity 0.0
aerial 0.7373263828065234
artificial 0.20179654138708839
follow 0.17151564160595856
geological 0.49854328442854257
hu 0.0
human 0.0
like 0.0
natgeo 0.0
national 0.20246822765264372
natural 0.20355105281214794
photo 0.07928323881789386
water 0.12061556784964586
world 0.0
years 0.1854847105894998
-------post 106 tf-idf------
activity 0.0
aerial 0.0
artificial 0.10166016914577468
follow 0.0
geological 0.37673089642510466
hu 0.1139958890471228
human 0.20597939903426757
like 0.0
natgeo 0.190830690762606
national 0.20399709656401413
natural 0.8203523932152975
photo 0.07988191882031571
water 0.0
world 0.0
years 0.18688533433596408
-------post 107 tf-idf------
activity 0.0
aerial 0.41295950239002543
artificial 0.0
follow 0.0
geological 0.13961129796675328
hu 0.063367967097066
human 0.17174955899437763
like 0.0
natgeo 0.0
national 0.0
natural 0.05700212513779502
photo 0.04440471358895925
water 0.8782019720644566
world 0.0
years 0.0
-------post 108 tf-id

human 0.20964001445625802
like 0.0
natgeo 0.12948139136805914
national 0.13841498866139804
natural 0.06957762458252348
photo 0.054201040475569146
water 0.0
world 0.2589627827361183
years 0.2536088196233398
-------post 144 tf-idf------
activity 0.17690355784503758
aerial 0.053006510216690755
artificial 0.30465089046762284
follow 0.07398171952971443
geological 0.053760533291113174
hu 0.4392232117161349
human 0.440907386843437
like 0.0
natgeo 0.08169619057842865
national 0.08733283735304455
natural 0.6584992813828849
photo 0.034198107430389715
water 0.05202643343789199
world 0.0
years 0.16001430198895517
-------post 145 tf-idf------
activity 0.0
aerial 0.18038721370462532
artificial 0.2962176589761241
follow 0.0
geological 0.0
hu 0.6643230218911
human 0.6001832958958403
like 0.0
natgeo 0.0
national 0.0
natural 0.07469827769555341
photo 0.05819003447049202
water 0.17705190038893293
world 0.13901073781767675
years 0.13613674042433568
-------post 146 tf-idf------
activity 0.0
aerial 0.483629

geological 0.23160806238751738
hu 0.5256212172843794
human 0.6648222172976725
like 0.18384670712300724
natgeo 0.0
national 0.0
natural 0.2836909035459172
photo 0.07366516768335353
water 0.11206865616672722
world 0.1759794336660627
years 0.17234112167968038
-------post 178 tf-idf------
activity 0.5999326190374112
aerial 0.0
artificial 0.1771134595305524
follow 0.0
geological 0.0
hu 0.29790732866741587
human 0.7177191275900913
like 0.0
natgeo 0.0
national 0.0
natural 0.0
photo 0.06958557670663144
water 0.0
world 0.0
years 0.0
-------post 179 tf-idf------
activity 0.30714439493609885
aerial 0.184062465520801
artificial 0.07556321692958663
follow 0.0
geological 0.09334038653764787
hu 0.677858096481977
human 0.6124115725179927
like 0.0
natgeo 0.0
national 0.0
natural 0.0
photo 0.0593756120149248
water 0.0
world 0.0
years 0.13891042261731001
-------post 180 tf-idf------
activity 0.0
aerial 0.47573887369379025
artificial 0.0781220866797496
follow 0.0
geological 0.5790075698823494
hu 0.1752032

aerial 0.1870352879804909
artificial 0.23035095176795348
follow 0.0
geological 0.09484794212053177
hu 0.34440314576544157
human 0.31115136512051
like 0.3011550056850378
natgeo 0.0
national 0.0
natural 0.23235372829659062
photo 0.06033459706631136
water 0.7343082139061424
world 0.0
years 0.0
-------post 218 tf-idf------
activity 0.0
aerial 0.0
artificial 0.10559208507961754
follow 0.0
geological 0.0
hu 0.11840491429568638
human 0.42789215108007445
like 0.20707263029553835
natgeo 0.0
national 0.0
natural 0.8520812076895691
photo 0.0829715161726982
water 0.12622663614314136
world 0.0
years 0.0
-------post 219 tf-idf------
activity 0.3416918106403897
aerial 0.0
artificial 0.0
follow 0.0
geological 0.0
hu 0.5655773737765004
human 0.6387143021759871
like 0.0
natgeo 0.0
national 0.0
natural 0.3815702993800898
photo 0.09908121739360667
water 0.0
world 0.0
years 0.0
-------post 220 tf-idf------
activity 0.0
aerial 0.23360343054601765
artificial 0.09590128355555204
follow 0.0
geological 0.947705

In [241]:
df2=pd.DataFrame(c)
df2.columns=z

In [244]:
df2.drop(["hu","follow","years","photo","world","like","natgeo"],axis=1,inplace=True)

In [245]:
logictic_pred(df2,y)

confusion_matrix:
 [[15  8]
 [ 1 24]]
accuracy_score:
 0.8125
